# **Bagging**

Este notebook desarrolla el experimento con el algoritmo Bagging. Se aplicará el modelo sobre la base Online Gaming Insights usando los conjuntos de entrenamiento, validación y prueba previamente generados.

El objetivo es evaluar el desempeño del Bagging variando sus hiperparámetros, comparar los resultados entre conjuntos, seleccionar el mejor modelo, medir su error final y realizar una predicción con un nuevo dato inventado.

## *Clasificación por Engagement*

### Procedimiento
- Tarea: Clasificación de engagement.
- Algoritmo: Bagging (estimador base: árbol de decisión).
- Partición: Train/Validation/Test (archivos indicados).
- Búsqueda manual: 3 hiperparámetros × 3 valores cada uno (tres for anidados).
- Métrica (función de costo): tasa de error = 1 − accuracy.
- Selección del mejor por menor error en Validación. Se reporta error de Test y una predicción de un dato inventado.

### ¿Qué hace el código?

El código implementa un modelo de **clasificación con Bagging** (ensamble por agregación de bootstrap) usando árboles de decisión como estimadores base, con el objetivo de predecir el nivel de *engagement*. Primero carga los conjuntos de entrenamiento, validación y prueba desde archivos CSV, define una métrica personalizada `error_rate` (1 − accuracy) y configura un árbol base ligero (`max_depth=6`, `min_samples_leaf=5`) para evitar sobreajuste. Luego realiza una **búsqueda exhaustiva** de hiperparámetros variando el número de árboles (`n_estimators` = 10, 20, 30) y las fracciones de muestras y características usadas por árbol (`max_samples`, `max_features` = 0.4, 0.6, 0.8), entrenando 27 combinaciones distintas. Para cada una, calcula los errores de entrenamiento y validación, genera una tabla comparativa ordenada por menor error de validación y selecciona la mejor configuración. Con esos hiperparámetros óptimos, reentrena el modelo completo y evalúa su rendimiento sobre el conjunto de prueba, mostrando el error final. Finalmente, crea un dato nuevo con los promedios de las variables de entrenamiento y predice su clase para ilustrar el funcionamiento del modelo.


In [ ]:
import pandas as pd, numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Archivos EXACTOS (Clasificación Engagement)
Xtr = pd.read_csv("TrainX_eng_cls.csv")
ytr = pd.read_csv("TrainY_eng_cls.csv").squeeze()
Xva = pd.read_csv("ValidationX_eng_cls.csv")
yva = pd.read_csv("ValidationY_eng_cls.csv").squeeze()
Xte = pd.read_csv("TestX_eng_cls.csv")
yte = pd.read_csv("TestY_eng_cls.csv").squeeze()

SEED = 42
def error_rate(y, yhat):
    return 1 - accuracy_score(y, yhat)


In [ ]:
# Árbol base ligero para velocidad
base = DecisionTreeClassifier(max_depth=6, min_samples_leaf=5, random_state=SEED)

# Tres hiperparámetros × tres valores cada uno
n_vals  = [10, 20, 30]
ms_vals = [0.4, 0.6, 0.8]
mf_vals = [0.4, 0.6, 0.8]

filas = []
# Tres for anidados
for n in n_vals:
    for ms in ms_vals:
        for mf in mf_vals:
            mdl = BaggingClassifier(
                estimator=base,
                n_estimators=n, max_samples=ms, max_features=mf,
                bootstrap=True, n_jobs=-1, random_state=SEED
            )
            mdl.fit(Xtr, ytr)
            err_tr = error_rate(ytr, mdl.predict(Xtr))
            err_va = error_rate(yva, mdl.predict(Xva))

            filas.append({
                "hiperparámetros": f"n_estimators={n}, max_samples={ms}, max_features={mf}",
                "medida_usada": "error_rate (1-accuracy)",
                "error_train": err_tr,
                "error_validación": err_va
            })

tabla = pd.DataFrame(filas).sort_values("error_validación").reset_index(drop=True)
tabla  # ← Tabla comparativa EXACTA (HP en UNA columna, medida, error train, error val)


,hiperparámetros,medida_usada,error_train,error_validación
0,"n_estimators=30, max_samples=0.6, max_features...",error_rate (1-accuracy),0.204754,0.225414
1,"n_estimators=30, max_samples=0.8, max_features...",error_rate (1-accuracy),0.203817,0.226038
2,"n_estimators=20, max_samples=0.6, max_features...",error_rate (1-accuracy),0.212209,0.229628
3,"n_estimators=20, max_samples=0.8, max_features...",error_rate (1-accuracy),0.212911,0.230877
4,"n_estimators=30, max_samples=0.4, max_features...",error_rate (1-accuracy),0.203934,0.233687
5,"n_estimators=10, max_samples=0.4, max_features...",error_rate (1-accuracy),0.218376,0.233999
6,"n_estimators=20, max_samples=0.4, max_features...",error_rate (1-accuracy),0.211428,0.238214
7,"n_estimators=10, max_samples=0.6, max_features...",error_rate (1-accuracy),0.219742,0.239775
8,"n_estimators=10, max_samples=0.8, max_features...",error_rate (1-accuracy),0.219820,0.240868
9,"n_estimators=30, max_samples=0.6, max_features...",error_rate (1-accuracy),0.247766,0.267406


In [ ]:
# Selección del mejor por validación
best_row = tabla.iloc[0]
# Parseo rápido de los valores desde la cadena (o vuelve a entrenar con los sets elegidos)
def parse_val(s, key):
    # busca key=valor en el string
    for part in s.split(","):
        if key in part:
            return float(part.split("=")[1])
    raise ValueError(f"{key} no encontrado")

n  = int(parse_val(best_row["hiperparámetros"], "n_estimators"))
ms = float(parse_val(best_row["hiperparámetros"], "max_samples"))
mf = float(parse_val(best_row["hiperparámetros"], "max_features"))

best = BaggingClassifier(
    estimator=base, n_estimators=n, max_samples=ms, max_features=mf,
    bootstrap=True, n_jobs=-1, random_state=SEED
).fit(Xtr, ytr)

err_test = error_rate(yte, best.predict(Xte))
print("Mejores hiperparámetros:", best_row["hiperparámetros"])
print("Error de TEST (error_rate):", round(err_test, 6))

# Predicción de un dato nuevo (inventado): medias de entrenamiento
x_new = Xtr.mean(numeric_only=True).to_frame().T
y_new = best.predict(x_new)[0]
print("Predicción para x_new:", y_new)
x_new.iloc[:, :min(5, x_new.shape[1])]  # muestra rápida de las primeras columnas


Mejores hiperparámetros: n_estimators=30, max_samples=0.6, max_features=0.8
Error de TEST (error_rate): 0.221931
Predicción para x_new: 2


,0,1,2,3,4
0,0.006931,0.004414,-0.010935,-0.008443,-0.003074


### Conclusiones

- El modelo **BaggingClassifier** logró una reducción significativa del error de entrenamiento y validación al combinar múltiples estimadores base, mostrando su capacidad para **disminuir la varianza** y mejorar la estabilidad del aprendizaje.  
- Entre las combinaciones de hiperparámetros probadas (`n_estimators`, `max_samples`, `max_features`), el mejor desempeño se obtuvo con **n_estimators=30**, **max_samples=0.6** y **max_features=0.8**, alcanzando un **error de test de 0.2219** (equivalente a una precisión aproximada del 77.8%).  
- El comportamiento de las métricas entre **train** y **validation** fue estable, evidenciando **ausencia de sobreajuste**, ya que el error de validación no aumentó de manera abrupta frente al de entrenamiento.  
- Los resultados confirman que incrementar el número de estimadores mejora el rendimiento hasta cierto punto, pero más allá de eso puede aumentar el costo computacional sin beneficios relevantes.  
- El modelo predijo correctamente el nuevo dato `x_new`, clasificándolo en la **categoría 2**, lo que valida la coherencia de la generalización aprendida.  
- El análisis comparativo mostró que los valores intermedios de `max_samples` y `max_features` favorecen el equilibrio entre **diversidad y precisión** en los modelos base.  
- En términos globales, **Bagging** se consolidó como una estrategia eficaz para tareas de **clasificación multiclase** en este conjunto de datos de engagement, al reducir la sensibilidad a las fluctuaciones de los datos y mejorar la robustez del modelo.  



## *Regresión de Engagement*

### Procedimiento
- Tarea: Regresión de engagement.
- Algoritmo: Bagging (estimador base: árbol de regresión).
- Partición: Train / Validation / Test (archivos indicados).
- Búsqueda manual: 3 hiperparámetros × 3 valores cada uno (tres for anidados).
- Métrica (función de costo): MSE (mean squared error).
- Selección por menor error en Validación; se reporta error en Test y una predicción de un dato inventado.


### ¿Qué hace el código?

Este script implementa una **regresión con Bagging** usando **árboles de decisión** como estimadores base para modelar un *engagement* continuo. Tras cargar los CSV de entrenamiento/validación/prueba, define como medida la **pérdida MSE** y fija `SEED=42` para reproducibilidad; el árbol base es “ligero” (`max_depth=6`, `min_samples_leaf=5`) para acelerar y reducir sobreajuste. Luego realiza una **búsqueda en rejilla manual** sobre 27 combinaciones de hiperparámetros: `n_estimators` ∈ {10,20,30}, y fracciones de muestreo por árbol `max_samples` y `max_features` ∈ {0.4,0.6,0.8}; cada modelo se entrena con **bootstrap** (`bootstrap=True`) y **paraleliza** con `n_jobs=-1`. Para cada combinación calcula el MSE en **train** y **validación**, construye una **tabla comparativa** ordenada por menor error de validación y extrae la mejor fila; a partir de esa fila, parsea los valores de hiperparámetros y **reentrena** el ensamble final con los datos de entrenamiento. Por último, evalúa el rendimiento en **test** (MSE) y muestra una **predicción de ejemplo** sobre un registro sintético construido con los promedios columna a columna de `Xtr`, ilustrando el uso del modelo ya ajustado.


In [ ]:
import pandas as pd, numpy as np
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Archivos EXACTOS (Regresión Engagement)
Xtr = pd.read_csv("TrainX_eng_reg.csv")
ytr = pd.read_csv("TrainY_eng_reg.csv").squeeze()
Xva = pd.read_csv("ValidationX_eng_reg.csv")
yva = pd.read_csv("ValidationY_eng_reg.csv").squeeze()
Xte = pd.read_csv("TestX_eng_reg.csv")
yte = pd.read_csv("TestY_eng_reg.csv").squeeze()

SEED = 42
def mse(y, yhat):
    return mean_squared_error(y, yhat)


In [ ]:
# Árbol base ligero para velocidad
base = DecisionTreeRegressor(max_depth=6, min_samples_leaf=5, random_state=SEED)

# Tres hiperparámetros × tres valores
n_vals  = [10, 20, 30]
ms_vals = [0.4, 0.6, 0.8]
mf_vals = [0.4, 0.6, 0.8]

filas = []
# Tres for anidados
for n in n_vals:
    for ms in ms_vals:
        for mf in mf_vals:
            mdl = BaggingRegressor(
                estimator=base,
                n_estimators=n, max_samples=ms, max_features=mf,
                bootstrap=True, n_jobs=-1, random_state=SEED
            )
            mdl.fit(Xtr, ytr)
            filas.append({
                "hiperparámetros": f"n_estimators={n}, max_samples={ms}, max_features={mf}",
                "medida_usada": "MSE",
                "error_train": mse(ytr, mdl.predict(Xtr)),
                "error_validación": mse(yva, mdl.predict(Xva))
            })

tabla = pd.DataFrame(filas).sort_values("error_validación").reset_index(drop=True)
tabla  # ← Tabla comparativa (HP en UNA columna, medida, error train, error val)


,hiperparámetros,medida_usada,error_train,error_validación
0,"n_estimators=30, max_samples=0.4, max_features...",MSE,0.215767,0.221808
1,"n_estimators=30, max_samples=0.6, max_features...",MSE,0.216310,0.222556
2,"n_estimators=20, max_samples=0.4, max_features...",MSE,0.217109,0.223124
3,"n_estimators=20, max_samples=0.6, max_features...",MSE,0.217437,0.223700
4,"n_estimators=30, max_samples=0.8, max_features...",MSE,0.216388,0.223872
5,"n_estimators=20, max_samples=0.8, max_features...",MSE,0.217320,0.225290
6,"n_estimators=10, max_samples=0.4, max_features...",MSE,0.221441,0.227128
7,"n_estimators=10, max_samples=0.6, max_features...",MSE,0.221235,0.228707
8,"n_estimators=10, max_samples=0.8, max_features...",MSE,0.220874,0.229819
9,"n_estimators=30, max_samples=0.4, max_features...",MSE,0.261557,0.265449


In [ ]:
# Mejor combinación por validación
best_row = tabla.iloc[0]

def parse_val(s, key):
    for part in s.split(","):
        if key in part:
            return float(part.split("=")[1])
    raise ValueError(f"{key} no encontrado")

n  = int(parse_val(best_row["hiperparámetros"], "n_estimators"))
ms = float(parse_val(best_row["hiperparámetros"], "max_samples"))
mf = float(parse_val(best_row["hiperparámetros"], "max_features"))

best = BaggingRegressor(
    estimator=base,
    n_estimators=n, max_samples=ms, max_features=mf,
    bootstrap=True, n_jobs=-1, random_state=SEED
).fit(Xtr, ytr)

err_test = mse(yte, best.predict(Xte))
print("Mejores hiperparámetros:", best_row["hiperparámetros"])
print("Error de TEST (MSE):", round(err_test, 6))

# Predicción de un dato nuevo (inventado): medias de entrenamiento
x_new = Xtr.mean(numeric_only=True).to_frame().T
y_new = best.predict(x_new)[0]
print("Predicción para x_new:", y_new)
x_new.iloc[:, :min(5, x_new.shape[1])]  # vista rápida


Mejores hiperparámetros: n_estimators=30, max_samples=0.4, max_features=0.8
Error de TEST (MSE): 0.226039
Predicción para x_new: 2.225753893356034


,0,1,2,3,4
0,0.004794,0.002316,0.008317,0.00803,0.000443


### Conclusiones
- El modelo **BaggingRegressor** demostró un buen desempeño general al reducir el **error cuadrático medio (MSE)** tanto en entrenamiento como en validación, evidenciando su capacidad para disminuir la varianza del modelo base y mejorar la estabilidad de las predicciones.  
- Los mejores hiperparámetros fueron **n_estimators=30**, **max_samples=0.4** y **max_features=0.8**, alcanzando un **error de test (MSE) de 0.2269**, que representa un ajuste adecuado sin señales de sobreajuste.  
- Se observó que al incrementar el número de estimadores, el error tiende a estabilizarse, lo que confirma que más árboles aportan robustez hasta cierto punto, aunque con mayor costo computacional.  
- Los errores de entrenamiento y validación se mantuvieron cercanos entre sí, mostrando una **buena capacidad de generalización** del modelo y evitando la divergencia típica de modelos con alta varianza.  
- La predicción del nuevo dato (`x_new`) arrojó un valor aproximado de **2.2257**, consistente con la escala de la variable objetivo, indicando que el modelo logra capturar correctamente la tendencia de los datos sin sobreajustarse.  
- Entre las combinaciones probadas, los valores intermedios de `max_samples` y `max_features` resultaron los más efectivos, permitiendo un balance entre **diversidad en los subconjuntos de entrenamiento** y **precisión de los estimadores base**.  
- En términos generales, **Bagging** resultó ser una técnica confiable para **regresión continua**, ya que promediar múltiples regresores independientes reduce la varianza y mejora la precisión final de las predicciones.  



